<a href="https://colab.research.google.com/gist/MaxGhenis/f14b2395ceb828d22d0dc07df033246b/openfisca-us-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenFisca US tutorial

OpenFisca US is [PolicyEngine](https://policyengine.org)'s open source tax and benefit microsimulation model.

This tutorial will teach you how to construct a household, calculate variables, and simulate a range of scenarios, using California's SNAP program as a case study. 

*See https://openfisca.us for full documentation.*

## Installation

OpenFisca US is available on PyPI.

In [1]:
#This code may take a little while to run
!pip install openfisca-us

     |████████████████████████████████| 377 kB 13.1 MB/s            
     |████████████████████████████████| 214 kB 40.5 MB/s            
     |████████████████████████████████| 596 kB 37.8 MB/s            
     |████████████████████████████████| 76 kB 248 kB/s             
     |████████████████████████████████| 11.3 MB 49.1 MB/s            
     |████████████████████████████████| 15.3 MB 18.8 MB/s            ��████████████████████████▏    | 13.0 MB 18.8 MB/s eta 0:00:01
     |████████████████████████████████| 248 kB 24.2 MB/s            
     |████████████████████████████████| 379 kB 27.3 MB/s            
     |████████████████████████████████| 94 kB 960 kB/s             
     |████████████████████████████████| 298 kB 27.6 MB/s            
     |████████████████████████████████| 79 kB 2.3 MB/s            
     |████████████████████████████████| 125 kB 36.2 MB/s            
     |████████████████████████████████| 82 kB 492 kB/s             
     |████████████████████████████████| 45 k

# Constructing households

To calculate taxes and benefits, first define the household via:
1. The `IndividualSim` constructor.
2. The `add_person` method.

Let's start with a single person with \$1,000 monthly employment income (inputs and outputs are all annual).

In [2]:
from openfisca_us import IndividualSim

sim = IndividualSim(year=2022)
sim.add_person(name="person", employment_income=1000 * 12)

## Calculating variables

Now we can calculate any variables in the [`openfisca_us/variables` folder](https://github.com/PolicyEngine/openfisca-us/tree/master/openfisca_us/variables) via the `calc` method.

For example, let's recover `employment_income`:

In [3]:
sim.calc("employment_income")

array([12000.], dtype=float32)

**Exercise 1: Calculate SNAP**

Calculate three SNAP-related variables:
1. `snap_normal_allotment`, which they would receive outside of Covid.
2. `snap_emergency_allotment`, which bumps up eligible households to the maximum allotment during the pandemic.
3. `snap`, which sums the two.

Print them as monthly amounts.

In [4]:
# Write your solution here.

## Households with multiple people

To create households with multiple people, we need to assign them to a household unit.
US tax and benefit programs group people in different ways, and SNAP uses a *SPM unit*, or a group of people that cohabit and share resources (SPM stands for the Supplemental Poverty Measure).

Let's now model a two-person household, still with \$1,000 monthly employment income, and group them with the `add_spm_unit` method.
This also enables us to add SPM-unit-level characteristics, like housing costs (which affect SNAP benefits).

In [5]:
sim = IndividualSim(year=2022)
sim.add_person(name="parent", employment_income=1000 * 12)
sim.add_person(name="child")
sim.add_spm_unit(
    name="spm_unit", members=["parent", "child"], housing_cost=600 * 12
)

Now let's recalculate SNAP.

In [6]:
print(
    "SNAP normal allotment: ",
    round(sim.calc("snap_normal_allotment")[0] / 12),
)
print(
    "SNAP emergency allotment: ",
    round(sim.calc("snap_emergency_allotment")[0] / 12),
)
print("Total SNAP: ", round(sim.calc("snap")[0] / 12))

SNAP normal allotment:  354
SNAP emergency allotment:  105
Total SNAP:  459


**Exercise 2: Customize a household**

Create a new household with a different number of people, or different income or housing costs.
Recalculate SNAP.

In [7]:
# Write your solution here.

## Scenario analysis

Suppose we want to explore a range of inputs, rather than manually specifying one or two.
We can apply techniques like list comprehension, but openfisca-us makes it easier with the `vary` method.

When we call `sim.vary`, subsequent `calc` calls calculate over the range of inputs specified in `vary`.

Let's vary employment income, going up to \$3,000 per month in increments of \$10 per month.

In [8]:
sim.vary("employment_income", max=3000 * 12, step=10 * 12)

Now when we call `calc`, we get an array rather than a scalar. Let's start by recovering employment income again.

In [9]:
sim.calc("employment_income")

array([[    0.,   120.,   240.,   360.,   480.,   600.,   720.,   840.,
          960.,  1080.,  1200.,  1320.,  1440.,  1560.,  1680.,  1800.,
         1920.,  2040.,  2160.,  2280.,  2400.,  2520.,  2640.,  2760.,
         2880.,  3000.,  3120.,  3240.,  3360.,  3480.,  3600.,  3720.,
         3840.,  3960.,  4080.,  4200.,  4320.,  4440.,  4560.,  4680.,
         4800.,  4920.,  5040.,  5160.,  5280.,  5400.,  5520.,  5640.,
         5760.,  5880.,  6000.,  6120.,  6240.,  6360.,  6480.,  6600.,
         6720.,  6840.,  6960.,  7080.,  7200.,  7320.,  7440.,  7560.,
         7680.,  7800.,  7920.,  8040.,  8160.,  8280.,  8400.,  8520.,
         8640.,  8760.,  8880.,  9000.,  9120.,  9240.,  9360.,  9480.,
         9600.,  9720.,  9840.,  9960., 10080., 10200., 10320., 10440.,
        10560., 10680., 10800., 10920., 11040., 11160., 11280., 11400.,
        11520., 11640., 11760., 11880., 12000., 12120., 12240., 12360.,
        12480., 12600., 12720., 12840., 12960., 13080., 13200., 

Since `employment_income` is person-level, `calc` returns a two-row array. `vary` varies the first person's variable by default, so the child's income stays fixed at zero.

### Calculating SNAP as income varies

Now we can calculate normal SNAP allotments.
Each value corresponds to the varied employment income above.

For example, at \$0 employment income, the household will get \$5,508 in annual benefits.
At \$3,000 monthly employment income, they are not eligible for any SNAP benefits.
Since `snap_normal_allotment` is at the SPM unit level, `calc` now returns a one-row array.

In [10]:
sim.calc("snap_normal_allotment")

array([[5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5508.    ,
        5508.    , 5508.    , 5508.    , 5508.    , 5508.    , 5502.6   ,
        5459.4   , 5416.2   , 5373.    , 5329.8   , 5286.6   , 5243.4   ,
        5200.2   , 5157.    , 5113.8  

**Exercise 3: Visualization**

Use your preferred data visualization library to create a plot of employment income on the x axis and normal (non-Covid) SNAP benefits on the y axis.

*Hint: `IndividualSim.calc` returns a list of all households' values. Since we're only working with one household, extract the first with `calc(x)[0]`.*

In [11]:
# Write your solution here.

## Learn more

Visit [openfisca.us](https://openfisca.us) for examples of other programs, calculating marginal tax rates, and more.